In [9]:
import os
from PIL import Image
import pprint

DATASET_PATH = "TestDatasetRobot"


image_paths= [os.path.join(DATASET_PATH, "Images", im_name) for im_name in os.listdir(os.path.join(DATASET_PATH, "Images"))]
label_paths = [os.path.join(DATASET_PATH, "Labels", lb_name) for lb_name in os.listdir(os.path.join(DATASET_PATH, "Labels"))]

In [10]:
class_mapping =  {
    0:'blue_armor', 
    1:'grey_armor', 
    2:'purple_armor', 
    3:'red_armor', 
    4:'robot'
}

In [11]:
pprint.pprint(sorted(image_paths))
pprint.pprint(sorted(label_paths))

image_paths = sorted(image_paths)
label_paths = sorted(label_paths)

['TestDatasetRobot/Images/-1_mp4-152_jpg.rf.c13e36e586ca61f687bf66fe7caf3b29.jpg',
 'TestDatasetRobot/Images/-1_mp4-57_jpg.rf.4344bd7729714128dae6712fe66bedff.jpg',
 'TestDatasetRobot/Images/-2_mp4-124_jpg.rf.e964b65faa070a14605cad586b9e2bfb.jpg',
 'TestDatasetRobot/Images/-_202307052203051_jpg.rf.a2195ba527c583b4ad87fedb3691d18f.jpg',
 'TestDatasetRobot/Images/-_202307052203052_jpg.rf.6f0d796dcf0e0c4354fe8334477bdc8d.jpg',
 'TestDatasetRobot/Images/-_202307052203054_jpg.rf.12276751346b98ed44abb21021c86084.jpg']
['TestDatasetRobot/Labels/-1_mp4-152_jpg.rf.c13e36e586ca61f687bf66fe7caf3b29.txt',
 'TestDatasetRobot/Labels/-1_mp4-57_jpg.rf.4344bd7729714128dae6712fe66bedff.txt',
 'TestDatasetRobot/Labels/-2_mp4-124_jpg.rf.e964b65faa070a14605cad586b9e2bfb.txt',
 'TestDatasetRobot/Labels/-_202307052203051_jpg.rf.a2195ba527c583b4ad87fedb3691d18f.txt',
 'TestDatasetRobot/Labels/-_202307052203052_jpg.rf.6f0d796dcf0e0c4354fe8334477bdc8d.txt',
 'TestDatasetRobot/Labels/-_202307052203054_jpg.rf.122

In [12]:
def convert_yolo_to_keylabs(x_center, y_center, width, height, image_width, image_height):
    x1 = (x_center - width / 2) * image_width
    y1 = (y_center - height / 2) * image_height
    x2 = (x_center + width / 2) * image_width
    y2 = (y_center + height / 2) * image_height

    return x1, y1, x2, y2

In [13]:
outoput_data = [{
        "version":"1.5",
        "file":"Libri-di-filosofia-da-leggere.jpg",
        "width":1920,
        "height":1280,
        "part":1,
        "original_frame":1,
        "frame_skip":1,
        "status":"open",
        "stage":"open",
        "objects":[]
}]

In [14]:
import json

# Create the annotations directory if it doesn't exist
if not os.path.exists('annotations'):
    os.makedirs('annotations')

for image_path, label_path in zip(image_paths, label_paths):
    img = Image.open(image_path)
    image_width, image_height = img.size

    metadata_objects = []
    data_objects = []

    
    with open(label_path, "r") as label_file:
        counter = 1
        for line in label_file.readlines():
            line = line.replace("\n", "").split(" ")
            print(line)
            label = line[0]

            nm = "0." + str(counter)
            counter += 1
            metadata_objects.append({
                 "nm" : nm,
                 "type": class_mapping[int(label)],
                 "shape" : "bBox",
             })
            
            x_center, y_center, width, height = line[1], line[2], line[3], line[4]
            x1, y1, x2, y2 = convert_yolo_to_keylabs(float(x_center), float(y_center), float(width), float(height), float(image_width), float(image_height))
            data_objects.append({
                "nm": nm,
                "key":True,
                "attr_key":False,
                "x1":x1,
                "y1":y1,
                "x2":x2,
                "y2":y2, 
                "path":[[
                    [x1, y2],
                    [x2, y2],
                    [x2, y1],
                    [x1, y1]
                ]
                ]
            })
            
            
            
    complete_data = [
       { "version": "1.5",
        "file": os.path.basename(image_path),
        "width": image_width,
        "height":image_height,
        "part":1,
        "original_frame":1,
        "frame_skip":1,
        "status":"open",
        "stage":"open",
        "objects": metadata_objects,
       },{
        "objects": data_objects
       }
    ]

    # Create a new JSON file for each image
    with open(f'annotations/{os.path.basename(image_path)}.json', 'w') as f:
        json.dump(complete_data, f)
            

['4', '0.8875', '0.64453125', '0.225', '0.4265625']
['1', '0.83125', '0.67109375', '0.046875', '0.090625']
['1', '0.9703125', '0.721875', '0.040625', '0.1']
['4', '0.81484375', '0.55', '0.084375', '0.1296875']
['0', '0.82109375', '0.57421875', '0.034375', '0.040625']
['0', '0.0953125', '0.49609375', '0.04375', '0.0515625']
['0', '0.5203125', '0.60625', '0.0515625', '0.0765625']
['0', '0.603125', '0.59140625', '0.0203125', '0.059375']
['3', '0.890625', '0.46328125', '0.1703125', '0.1421875']
['3', '0.40390625', '0.4546875', '0.115625', '0.0859375']
['3', '0.63203125', '0.4484375', '0.0859375', '0.09375']
['3', '0.52578125', '0.4390625', '0.1875', '0.0703125']
['4', '0.52890625', '0.49140625', '0.65625', '0.43984375']
['0', '0.46171875', '0.32890625', '0.1859375', '0.1']
